# <span style="color: blue">Laboratorio 6: Estimación II</span> <a class="tocSkip"> 

### <span style="color: green">Prof. Andrés Arguedas - XS-0300 - II 2020</span> <a class="tocSkip"> 

*Este laboratorio es parte del curso XS-0300 Estadística para Biología I de la Escuela de Estadística de la Universidad de Costa Rica, y se presenta con una licencia [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/).*

**ATENCIÓN:** Antes de entregar este laboratorio, asegúrese que todo corre como se espera. Para hacer esto, primero **reinice el kernel**, esto lo puede hacer seleccionado **Kernel** en el menú y luego **Restart**. Habiendo reiniciado el kernel, **corra todas las celdas**, esto lo puede hacer seleccionando **Cell** en el menú y luego **Run All**.

Por último, recuerde rellenar cualquier lugar donde diga `ESCRIBA SU CÓDIGO AQUÍ` o "ESCRIBA SU RESPUESTA AQUI", así como escribir su nombre y carné a continuación:

In [ ]:
NOMBRE = ""
CARNE = ""

---

## Preparación para este laboratorio
    
El presente laboratorio tiene como objetivos de aprendizaje:
    
1. Repasar el uso de los box plots para identificar diferencias en la distribución de las variables para distintos grupos
2. Conocer argumentos y usos nuevos para la función `t.test()`, para poder calcular intervalos de confianza para la diferencia de promedios
3. Recalcar el concepto de muestras pareadas, y la diferencia entre los intervalos de confianza asumiendo independencia o muestras pareadas

En este laboratorio utilizaremos los conjuntos de datos de `aves` y `venados`, junto con los paquetes `ggplot2`, `readr`, `dplyr` y `tidyr`. Ya todos estos paquetes están descargados y disponibles en el entorno virtual, al igual que el conjunto de datos, por lo que no es necesario que descarguen o consigan ningún paquete o conjunto de datos adicional, todos ya están disponibles.

---

## Intervalo de confianza para la diferencia de medias

En este laboratorio vamos a utilizar dos conjuntos de datos distintos para cada uno de los dos temas que veremos: `aves` para el caso de muestra independientes y `venados` para el caso de muestras pareadas. En cada una de las secciones del laboratorio explicaremos con mayor profunidad los conjuntos de datos, junto con las fórmulas y funciones para obtener los resultados que queremos en R.

### Muestras independendientes (`aves`)

El conjunto de datos de `aves` lo hemos trabajado en laboratorios anteriores, de forma más reciente, durante el primer laboratorio de Estimación. Por lo tanto, vamos a proceder directamente a cargar los datos, hacer los arreglos necesarios al conjunto de datos y hacer un summary:

In [ ]:
library(readr)
library(dplyr)

aves <- read_csv("data/aves.csv")

aves <- aves %>% mutate(Especie = factor(Especie),
                        Localidad = factor(Localidad)) %>% 
                 rename(Largo_pico = `Largo pico`, 
                        Alto_pico = `Alto pico`, 
                        Ancho_pico = `Ancho pico`) %>% 
                 na.omit()

summary(aves)

Recordemos que tenemos aves de dos especies en este conjunto de datos: *Catharus gracilirostris* y *Chlorospingus pileatus*. Además, ya en otros laboratorios habíamos planteado que algunas variables, específicamente `Juanete` y `Alto_pico` tenían una distribución bimodal, que podía ser separada según especie. Por lo tanto, para este caso específico, vamos a trabajar con la variable `Alto_pico`, separado según especie, para determinar si realmente los promedios de ambas especies son distintos. Por lo tanto, como primer acercamiento, vamos a hacer un box plot para la variable `Alto_pico`, separado según especie, para determinar visualmente si realmente parecen haber diferencias para esta variable según la especie:

In [ ]:
library(ggplot2)

aves %>% ggplot(aes(y = Alto_pico, x = Especie, fill = Especie)) +
    geom_boxplot() +
    guides(fill = F) + # Eliminar la leyenda
    labs(title = "Alto del pico según especie",
           y = "Alto del pico") +
    theme_bw()

Como podemos notar del gráfico anterior, las cajas correspondientes a ambas especies están centradas en puntos completamente distintos: cerca de 4 para *Catharus gracilirostris*, comparado con un valor cercano a 6,5 para *Chlorospingus pileatus*. Además del punto donde están centradas, también podemos notar que las cajas no se intersecan, es decir, están completamente separadas. Con respecto a la variabilidad, parece que la caja de la izquiera es un poco más estrecha que la de la derecha, es decir. En conclusión, este gráfico parece respaldar nuestra noción original de que el comportamiento del alto del pico de las aves varía según especie, donde el alto del pico de las aves de la especie *Catharus gracilirostris* son menores, en promedio, que los de la especie *Chlorospingus pileatus*.

Utilizando las funciones que ya conocemos, podemos calcular el promedio y la desviación estándar para esta variable, según cada especie, junto con el tamaño de muestra respectivo:

In [ ]:
aves %>% group_by(Especie) %>% 
    summarise(media = mean(Alto_pico),
             desv.est = sd(Alto_pico),
             n = n())

Estos resultados nos dicen lo mismo que ya intuimos del box plot, por lo que no vamos a interpretarlos nuevamente, pero si vale la pena resaltar que, con base en estos 6 valores, ya podemos calcular el intervalo de confianza para la diferencia de promedios. Para esto, primero recordemos la fórmula para dicho intervalo de confianza, en el caso de muestras independientes:

$$
\text{Intervalo de confianza } 100 (1 - \alpha) \% =  (\bar{x}_1 - \bar{x}_2) \pm t_{1 - \alpha / 2, n_1+n_2-2} \sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}
$$

Por lo tanto, dado un nivel de confianza dado, con los resultados que obtuvimos anteriormente ya podríamos calcular el intervalo de confianza en este caso. Aun así, dado que ya conocemos la función `t.test()` de R, podemos utilizar esta para calcular el intervalo de confianza deseado, como se muestra a continuación:

In [ ]:
t.test(aves$Alto_pico ~ aves$Especie)

La principal diferencia al momento de escribir el comando, comparado con el intervalo de confianza para una sola media, es que ahora utilizamos dos variables. Si `X` es nuestra variable numérica de interés (`Alto_pico` en nuestro caso) y `Y` es la variable que separa la muestra en dos grupos (`Especie` en nuestro caso), entonces el intervalo de confianza para la diferencia de promedios se puede escribir como `t.test(X ~ Y)`. El símbolo `~`, llamado virgulilla o tilde de la eñe, es un símbolo que, cuando es utilizado en R, se puede leer como **según**. Por ejemplo, `aves$Alto_pico ~ aves$Especie` se puede leer como **Alto del pico según especie**, lo cual es bastante explícito en lo que queremos hacer.

Al momento de interpretar este intervalo de confianza, es muy importante tener claro cual grupo se está tomando como el 1 y cual como el 2. En este caso, si vemos las últimas líneas del resultado del comando, nos devuelve lo siguiente:

```
sample estimates:
mean in group Catharus gracilirostris  mean in group Chlorospingus pileatus 
                             4.087838                              6.395455 
```

Como podemos ver, lo que nos dice es que el primer grupo es *Catharus gracilirostris*, que tiene un promedio menor que el del segundo grupo, *Chlorospingus pileatus*. Cabe resaltar que R asigna los grupos de forma ordenada, según los niveles del factor, por lo que la forma más fácil de determinar cuál grupo es cual en el comando es conociendo las etiquetas de la variable de factor. Por lo tanto, la interpretación de este intervalo de confianza sería que, **con 95% de confianza, la verdadera diferencia en la altura del pico promedio entre aves de las especies *Catharus gracilirostris* y *Chlorospingus pileatus* está entre -2.44 mm y -2.17 mm**. Aunque esta interpretación está correcta, se vuelve un poco difícil entender lo que realmente se quiere concluir sobre la relación entre ambas especies, en otras palabras, cual de las dos especies tiene mayor altura de pico. Para poder llegar a esta interpretación, es importante recordar los tres posibles casos para la diferencia de los promedios:

$$
\begin{align*}
    \bar{x}_1 - \bar{x}_2 < 0 \Rightarrow \bar{x}_1 < \bar{x}_2 & \Rightarrow \text{La población 1 tiene menor promedio que la 1} \\
    \bar{x}_1 - \bar{x}_2 = 0 \Rightarrow \bar{x}_1 = \bar{x}_2 & \Rightarrow \text{La población 1 tiene igual promedio que la 2} \\
    \bar{x}_1 - \bar{x}_2 > 0 \Rightarrow \bar{x}_1 > \bar{x}_2 & \Rightarrow \text{La población 1 tiene mayor promedio que la 2}
\end{align*}
$$

En otras palabras, si todo el intervalo es menor que 0, entonces podemos interpretarlo como que tan grande es el promedio de la población 2 con respecto al de la población 1; mientras que, si todo el intervalo es mayor que 0, podemos interpretarlo como que tan grande es el promedio de la población 1 con respecto a la població 2. El caso especial es cuando el intervalo de confianza contiene al 0, es decir, cuando tiene tantos números positivos como negativos. En esos casos no podemos realizar conclusiones sobre la diferencia en los promedios, ya que la población 1 puede tener mayor promedio que la población 2, o vice versa, pero también puede suceder que ambas poblaciones tienen el mismo promedio. Por lo tanto, como no conocemos el comportamiento de la diferencia, solo podemos decir que no podemos concluir sobre la diferencia de promedios. Habiendo dicho esto, podemos interpretar el intervalo de confianza que calculamos como que, **con un 95% de confianza, el alto del pico promedio de las aves de la especie *Chlorospingus pileatus* es entre 2,44 mm y 2,17 mm más grande que el de las aves de la especie *Catharus gracilirostris***. Esta interpretación, aunque de fondo es la misma que la primera que realizamos, se torna más fácil de entender, y nos da una idea clara sobre cuál grupo es el que tiene el promedio más grande, que es, al fin y al cabo, lo que queremos investigar.

Por otro lado, recordemos que podemos calcular el mismo intervalo de confianza, pero asumiendo que la desviación estándar es igual en ambos grupos, para facilitar el análisis y cálculos. En dicho caso, la fórmula para este intervalo de confianza es la siguiente:

$$
\text{Intervalo de confianza } 100 (1 - \alpha) \% =  (\bar{x}_1 - \bar{x}_2) \pm t_{1 - \alpha / 2, n_1+n_2-2} \sqrt{\frac{s_p^2}{n_1} + \frac{s_p^2}{n_2}}
$$

donde

$$
s_p^2 = \frac{(n_1 - 1) s_1^2 + (n_2 - 1) s_2^2}{n_1 + n_2 - 2}
$$

es el promedio ponderado de las variancias de ambos grupos, con respecto a su tamaño de muestra respectivo. Ahora, para hacer este cálculo utilizando R, es cuestión de agregar el argumento `var.equal = T` al comando `t.test()`, como se muestra a continuación:

In [ ]:
t.test(aves$Alto_pico ~ aves$Especie, var.equal = T)

En este caso, como la desviación estándar del alto del pico de ambas especies eran muy similares, como vimos al calcularlos, entonces el resultado es muy parecido, por lo que, sin importar si asumimos igualdad de variancia o no, el resultado va a ser, para fines prácticos, exactamente igual, en este caso.

---

#### Ejercicio (7 pts.)

Dado que ya vimos el comportamiento de la variable `Alto_pico` según la especie, vamos ahora a replicar dicho análisis, pero utilizando la variable `Juanete`.

1) Haga un gráfico de cajas del largo del juanete de las aves según su especie. ¿Parece haber alguna diferencia en esta variable según la especie? (2 pts.)

In [ ]:
# ESCRIBA SU CÓDIGO AQUÍ
testthat::fail() # No respuesta - elimine esta línea si escribió una respuesta

ESCRIBA SU RESPUESTA AQUI

2) Calcule, e interprete, el intervalo de confianza para la diferencia de promedios, utilizando la variable `Juanete` y separando según `Especie`, con un 95% de confianza. ¿Concuerda el resultado del intervalo de confianza con su hipótesis de la primer pregunta? (3 pts.)

In [ ]:
# ESCRIBA SU CÓDIGO AQUÍ
testthat::fail() # No respuesta - elimine esta línea si escribió una respuesta

ESCRIBA SU RESPUESTA AQUI

3) Manteniendo el mismo nivel de confianza de la pregunta 2) (95%), pero ahora duplicando el tamaño de muestra para cada grupo, ¿cree que las conclusiones que obtendría del nuevo intervalo de confianza serían las mismas que las de la pregunta 2)? Justifique su respuesta, aunque no es necesario que realice ningún cálculo. (2 pts.)

ESCRIBA SU RESPUESTA AQUI

### Muestras pareadas (`venados`)

Para esta sección utilizaremos el conjunto de datos `venados`, el cual contiene las mediciones de las patas traseras y delanteras de un grupo de 10 venados. Recordemos que el concepto de **muestras pareadas** aplica para aquellas variables que tienen una relación entre ellas como, por ejemplo, medir un mismo individuo varias veces, medir de distintas formas un mismo individuo, u observaciones que provienen de individuos idénticos (gemelos). En este caso, dado que estamos midiendo las patas traseras y delanteras de un mismo venado, tiene sentido sea una muestra pareada, dado que, de forma natural, las patas de los venados tienen que estar relacionados. Por lo tanto, para empezar, vamos a cargar el conjunto de datos y veremos que contiene:

In [ ]:
venados <- read_csv("data/venados.csv")

venados

Como podemos notar, el conjunto de datos tiene tres variables: `Venado`, `Trasera` y `Delantera` las cuales, como lo dice su nombre, determinan el venado, junto con el largo de sus patas traseras y delanteras, respectivamente. En otras palabras, cada fila del conjunto de datos corresponde a un venado distinto por lo que, al comparar el largo de las patas traseras con las delanteras, estaríamos trabajando con una muestra pareada, dado que provienen del mismo venado. Por ahora, vamos a cambiar la variable `Venado` a factor, para poder utilizarla más fácilmente en los procedimientos siguientes:

In [ ]:
venados <- venados %>% mutate(Venado = factor(Venado))

Habiendo hecho el cambio respectivo, nuestro primer acercamiento gráfico a este problema va a ser mediante un gráfico de dispersión, solamente que un poco distinto al usual. En este caso, al tratarse de muestras pareadas, lo que queremos hacer es comparar el largo de las patas, para cada venado individualmente. Es decir, necesitamos que el venado esté en el eje X y que las patas estén en el eje X. Aun así, esto no lo podemos hacer directamente con el conjunto de datos que tenemos, por lo menos utilizando ggplot, por lo que a continuación se presenta un código que generará el gráfico que queremos. Vaya corriendo el bloque de código línea por línea, para entender que es lo que hace cada comando.

In [ ]:
library(tidyr)

venados %>% pivot_longer(- Venado, names_to = "Pata", values_to = "Largo") %>% 
    ggplot(aes(x = Venado, y = Largo, group = Pata, col = Pata)) +
    geom_point() +
    geom_line() +
    theme_bw()

De primera entrada, podemos notar que la mayoría de los venados (8 de 10) tienen las patas traseras más grandes que las delanteras. Esto se puede ver en el gráfico anterior, donde en solamente dos puntos (es decir dos venados) el punto rojo (pata delantera) está por encima del punto azul (pata trasera). Por lo tanto, con base en lo anterior, podemos realizar la hipótesis que las patas traseras de los venados van a ser, en promedio, más grandes que las traseras. Lo anterior lo podemos comprobar mediante el intervalo de confianza para la diferencia de promedios, pero con muestras pareadas.

Recordemos que el procedimiento en el caso de muestras pareadas es distinto al de muestras independientes. En el caso de muestras pareadas, necesitamos calcular una nueva variable, correspondiente a la diferencia para cada observación, y calcular el intervalo de confianza para el promedio con base en dicha nueva variable. Por lo tanto, para empezar, vamos a crear una nueva variable, llamada `Diferencia`, que será igual a la altura de la pata trasera menos la pata delantera del venado:

In [ ]:
venados <- venados %>% mutate(Diferencia = Trasera - Delantera)

venados

Como podemos ver, ahora tenemos una nueva variable en `venados`, que contiene los valores de las diferencias entre las patas. Es importante resaltar que, dado que estamos restando la altura de la pata trasera menos la delantera, un valor positivo de la diferencia indica una pata trasera más grande que la delantera, mientras que un valor negativo indica una pata delantera más grande que la trasera. Por lo tanto, podemos crear un gráfico equivalente al anterior, pero utilizando la diferencia, como se muestra a continuación:

In [ ]:
venados %>% ggplot(aes(y = Diferencia, x = Venado)) +
geom_point() +
geom_hline(yintercept = 0, col = "red", lty = "dashed") +
theme_bw()

Aquí la línea roja punteada representa una diferencia de cero, es decir, que la altura de las patas traseras y delanteras son iguales, por lo que los puntos por encima de la línea tienen patas traseras más grandes y los puntos por debajo de la línea tiene las patas delanteras más grandes. De cierta forma, este gráfico presenta la misma información que el gráfico separado según las distintas patas, pero de forma un poco distinta. Por lo tanto, la interpretación y conclusiones van a ser las mismas, así que podemos proceder con el método de cálculo. Dado que solamente ocupamos calcular un intervalo de confianza para un promedio en este caso, lo podemos realizar directamente mediante la función `t.test()`:

In [ ]:
t.test(venados$Diferencia)

La forma de utilizar este comando es igual al caso de una sola variable, por lo que no pondremos mucha atención sobre eso. Lo que más nos interesa es la interpretación de este intervalo, donde ocupamos recordar lo que comentamos en la sección de muestras independendientes. En este caso, podemos interpretar este intervalo como que, **con un 95% de confianza, la diferencia verdadera entre las alturas promedio de las patas traseras y delanteras de los venados está entre 1.11 cm y 5.47 cm**. Lo cual lo podemos expresar, de igual forma, como que **con un 95% de confianza, la altura promedio verdadera de las patas traseras de los venados es entre 1.11 cm y 5.47 cm mayor que el de las patas delanteras**. En otras palabras, nuestra hipótesis original se mantiene: las patas traseras de los venados son, en promedio, más largas que las delanteras. Utilizando directamente la función `t.test()`, si agregamos el argumento `paired = T`, le podemos indicar a R que realice una diferencia de promedios, usando muestras pareadas, como se muestra a continuación:

In [ ]:
t.test(venados$Trasera, venados$Delantera, paired = T)

Primero, veamos que en este caso necesitamos darle dos variables a la función, correspondiente a las dos variables que queremos compararles el promedio. Luego, podemos confirmar que realmente es una diferencia de promedios para muestras pareadas dado que, en la primera linea del resultado dice

```
Paired t-test
```

lo que indica que estamos utilizando muestras pareadas. Por último, podemos ver que el resultado del comando utilizando `paired = T` y la variable `Diferencia` son iguales, por lo que cualquier de los dos que utilicemos será igual.

**Una última observación:** No debemos utilizar la fórmula para muestras pareadas cuando tengamos muestras independientes, ni vice versa. Aunque en el caso donde las unidades pareadas son muy parecidas los intervalos son muy similares, esto no siempre es el caso. Supongamos que en el caso de los venados partimos de que son muestras independientes y calculemos el intervalo de confianza para la diferencia de medias:

In [ ]:
t.test(venados$Trasera, venados$Delantera)

En este caso el intervalo de confianza es más ancho y, lo más problemático, contiene al 0. Por lo tanto, si asumimos que las patas de los venados son independientes, estamos calculando mal la variabilidad del intervalo de confianza, por lo que llegaríamos a una conclusión errónea en este caso: que no podemos discernir si las patas traseras tienen distinto tamaño, en promedio que las delantes. En conclusión, siempre se deben usar las fórmulas adecuadas para cada tipo de intervalo de confianza.

---

## <span style="color: blue">Entrega del laboratorio</span> <a class="tocSkip"> 

Para hacer entrega de este laboratorio, debe descargarlo presionando en **File**, luego **Download as** y, por último, **Notebook (.ipynb)**, en el menú superior, debajo del logo de Jupyter. También puede descargarlo directamente desde el botón de **Download** en la barra con íconos para acceso rápido.

Luego de haber localizado el archivo descargado, debe cambiar el nombre del archivo descargado por su número de carné, de forma que, si su carné es **B12345**, su archivo debería llamarse `B12345.ipynb`. Una vez le haya cambiado el nombre al archivo, lo puede subir a Mediación Virtual, en el apartado correspondiente a este laboratorio.

<a href="https://creativecommons.org/licenses/by-sa/4.0/">
    <img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-sa.png" width="88" height="31" />
</a>